## This script uses Scanpy to preprocess scRNA-seq data, then applies LIANA to perform communication inference for each sample, and constructs a 4-dimensional communication score tensor.

In [2]:
import pandas as pd
import scanpy as sc
import liana as li
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

# 1. Preprocessing Data

## 1.1. Download Data

**RNA-seq data**

Expression matrix was obtained from: https://cells.ucsc.edu/autism/exprMatrix.tsv.gz
<br>Values are 10x UMI counts from cellranger, log2-transformed

Metadata was obtained from: https://cells.ucsc.edu/autism/meta.tsv

Here is the bash script to download data:

```
raw_data_dir=/projects/compbio/users/rche384/APE/Data/RDA/ASD/raw
mkdir -p ${raw_data_dir}
cd ${raw_data_dir}
wget https://cells.ucsc.edu/autism/exprMatrix.tsv.gz 
wget https://cells.ucsc.edu/autism/meta.tsv
wget --no-check-certificate https://cells.ucsc.edu/autism/rawMatrix.zip
```

In [3]:
data_folder = '/projects/compbio/users/rche384/APE/Data/RDA/ASD/raw'
# Load the raw data
adata = sc.read_mtx(f'{data_folder}/matrix.mtx').T  # Transpose because we want data is in cells x genes format

# Load gene names
genes = pd.read_csv(f'{data_folder}/genes.tsv', header=None, sep='\t')
genes.columns = ['gene_id', 'gene_symbol']
genes['combined'] = genes['gene_id'] + '|' + genes['gene_symbol']
adata.var_names = genes['combined'].values

# Load cell barcodes
barcodes = pd.read_csv(f'{data_folder}/barcodes.tsv', header=None, sep='\t')
adata.obs_names = barcodes[0].values

# Load metadata
metadata = pd.read_csv(f'{data_folder}/meta.txt', sep='\t')

# Set metadata into adata.obs, ensuring that the barcodes match
adata.obs = metadata.set_index('cell')

In [4]:
print(adata)

AnnData object with n_obs × n_vars = 104559 × 65217
    obs: 'cluster', 'sample', 'individual', 'region', 'age', 'sex', 'diagnosis', 'Capbatch', 'Seqbatch', 'post-mortem interval (hours)', 'RNA Integrity Number', 'genes', 'UMIs', 'RNA mitochondr. percent', 'RNA ribosomal percent'


## 1.2 Select PFC brain region

In [6]:
brain_area = 'PFC'
filtered_adata = adata[adata.obs.region == brain_area]

In [7]:
print(filtered_adata)

View of AnnData object with n_obs × n_vars = 62166 × 65217
    obs: 'cluster', 'sample', 'individual', 'region', 'age', 'sex', 'diagnosis', 'Capbatch', 'Seqbatch', 'post-mortem interval (hours)', 'RNA Integrity Number', 'genes', 'UMIs', 'RNA mitochondr. percent', 'RNA ribosomal percent'


## 1.3 Basic Quality Control

In [8]:
# filter cells and genes
sc.pp.filter_cells(filtered_adata, min_genes=200)
sc.pp.filter_genes(filtered_adata, min_cells=3)
# log1p normalize the data
sc.pp.normalize_total(filtered_adata)
sc.pp.log1p(filtered_adata)

In [9]:
print(filtered_adata)

AnnData object with n_obs × n_vars = 62166 × 36745
    obs: 'cluster', 'sample', 'individual', 'region', 'age', 'sex', 'diagnosis', 'Capbatch', 'Seqbatch', 'post-mortem interval (hours)', 'RNA Integrity Number', 'genes', 'UMIs', 'RNA mitochondr. percent', 'RNA ribosomal percent', 'n_genes'
    var: 'n_cells'
    uns: 'log1p'


In [10]:
filtered_adata.obs.head()

cluster     sample  individual region  \
cell                                                                           
AAACCTGAGATGGCGT-1_4341_BA46            AST-PP  4341_BA46        4341    PFC   
AAACCTGAGCTAACTC-1_4341_BA46  Oligodendrocytes  4341_BA46        4341    PFC   
AAACCTGAGTGTTAGA-1_4341_BA46              L5/6  4341_BA46        4341    PFC   
AAACCTGCACCCATTC-1_4341_BA46              L2/3  4341_BA46        4341    PFC   
AAACCTGTCTCATTCA-1_4341_BA46         Microglia  4341_BA46        4341    PFC   

                              age sex diagnosis Capbatch Seqbatch  \
cell                                                                
AAACCTGAGATGGCGT-1_4341_BA46   13   M   Control      CB6      SB2   
AAACCTGAGCTAACTC-1_4341_BA46   13   M   Control      CB6      SB2   
AAACCTGAGTGTTAGA-1_4341_BA46   13   M   Control      CB6      SB2   
AAACCTGCACCCATTC-1_4341_BA46   13   M   Control      CB6      SB2   
AAACCTGTCTCATTCA-1_4341_BA46   13   M   Control      CB6      SB2   

                              post-mortem interval (hours)  \
cell                                                         
AAACCTGAGATGGCGT-1_4341_BA46                            16   
AAACCTGAGCTAACTC-1_4341_BA46                            16   
AAACCTGAGTGTTAGA-1_4341_BA46                            16   
AAACCTGCACCCATTC-1_4341_BA46                            16   
AAACCTGTCTCATTCA-1_4341_BA46                            16   

                              RNA Integrity Number  genes   UMIs  \
cell                                                               
AAACCTGAGATGGCGT-1_4341_BA46                   7.2    792   1121   
AAACCTGAGCTAACTC-1_4341_BA46                   7.2    658    928   
AAACCTGAGTGTTAGA-1_4341_BA46                   7.2   4890  13640   
AAACCTGCACCCATTC-1_4341_BA46                   7.2   3967   8526   
AAACCTGTCTCATTCA-1_4341_BA46                   7.2    621    846   

                              RNA mitochondr. percent  RNA ribosomal percent  \
cell                                                                           
AAACCTGAGATGGCGT-1_4341_BA46                 0.089206               1.338091   
AAACCTGAGCTAACTC-1_4341_BA46                 0.431034               0.862069   
AAACCTGAGTGTTAGA-1_4341_BA46                 0.857771               0.469208   
AAACCTGCACCCATTC-1_4341_BA46                 0.469153               0.516069   
AAACCTGTCTCATTCA-1_4341_BA46                 0.236407               0.945627   

                              n_genes  
cell                                   
AAACCTGAGATGGCGT-1_4341_BA46      792  
AAACCTGAGCTAACTC-1_4341_BA46      658  
AAACCTGAGTGTTAGA-1_4341_BA46     4890  
AAACCTGCACCCATTC-1_4341_BA46     3967  
AAACCTGTCTCATTCA-1_4341_BA46      621

In [11]:
filtered_adata.var.head()

n_cells
ENSG00000227232|WASH7P            1902
ENSG00000243485|RP11-34P13.3        34
ENSG00000238009|RP11-34P13.7       410
ENSG00000233750|CICP27             781
ENSG00000268903|RP11-34P13.15      198

In [12]:
filtered_adata.var.index = [idx.split('|')[1] for idx in filtered_adata.var.index]

In [13]:
filtered_adata.var.head()

n_cells
WASH7P            1902
RP11-34P13.3        34
RP11-34P13.7       410
CICP27             781
RP11-34P13.15      198

# 2. Use LIANA to Perform CCC Inference by Sample

In [14]:
sample_key = 'sample'
groupby = 'cluster'

In [15]:
li.mt.rank_aggregate.by_sample(
    filtered_adata,
    groupby=groupby,
    resource_name='consensus', # NOTE: uses human gene symbols!
    sample_key=sample_key, # sample key by which we which to loop
    use_raw=False,
    verbose=True, # use 'full' to show all verbose information
    n_perms=None, # exclude permutations for speed
    return_all_lrs=True, # return all LR values
    )

Converting `sample` to categorical!
Now running: 6033_BA9: 100%|██████████| 23/23 [09:46<00:00, 25.50s/it]     


- Check LIANA results

In [17]:
filtered_adata.uns["liana_res"].sort_values("magnitude_rank").head(10)

sample            source target ligand_complex  \
3988778        5387_BA9  Oligodendrocytes     L4       IL1RAPL1   
12063438       5841_BA9  Oligodendrocytes     L4       IL1RAPL1   
6003686   5408_PFC_Nova  Oligodendrocytes     L4       IL1RAPL1   
11025061       5577_BA9  Oligodendrocytes   L5/6       IL1RAPL1   
0             4341_BA46  Oligodendrocytes   L5/6       IL1RAPL1   
19066486       5958_BA9  Oligodendrocytes     L4       IL1RAPL1   
8988767   5538_PFC_Nova  Oligodendrocytes   L5/6       IL1RAPL1   
2965140        5294_BA9  Oligodendrocytes   L5/6       IL1RAPL1   
7958771        5531_BA9  Oligodendrocytes     L4       IL1RAPL1   
14121118  5879_PFC_Nova  Oligodendrocytes   L5/6       IL1RAPL1   

         receptor_complex  lr_means  expr_prod  scaled_weight  lr_logfc  \
3988778             PTPRD  2.749131   7.532037       1.341422  1.574887   
12063438            PTPRD  2.885699   8.323357       1.310877  1.806469   
6003686             PTPRD  2.774165   7.693921       1.410131  1.828380   
11025061            PTPRD  2.958418   8.747614       1.299729  2.050153   
0                   PTPRD  2.909284   8.453215       1.234929  1.877219   
19066486            PTPRD  2.847239   8.106462       1.261262  1.520966   
8988767             PTPRD  2.546762   6.485092       1.146841  1.639261   
2965140             PTPRD  2.939969   8.642257       1.423251  1.901086   
7958771             PTPRD  2.998587   8.989957       1.567351  1.936722   
14121118            PTPRD  2.683192   7.198975       1.341997  1.754207   

          spec_weight   lrscore  magnitude_rank  
3988778      0.013149  0.985969    2.748508e-11  
12063438     0.013902  0.987183    2.772891e-11  
6003686      0.016184  0.985670    2.855456e-11  
11025061     0.015614  0.987234    2.861518e-11  
0            0.018843  0.986918    2.867882e-11  
19066486     0.015193  0.985540    2.874307e-11  
8988767      0.017277  0.984700    2.912201e-11  
2965140      0.016388  0.986878    2.925911e-11  
7958771      0.015964  0.986922    3.002469e-11  
14121118     0.017267  0.984811    3.040049e-11

- Check number of CCC events by sample

In [18]:
filtered_adata.uns["liana_res"]['sample'].value_counts()

sample
5387_BA9         1076814
5841_BA9         1060341
4341_BA46        1052249
5958_BA9         1043290
5577_BA9         1038377
5531_BA9         1029996
5538_PFC_Nova    1028551
5408_PFC_Nova    1024216
5294_BA9         1023638
5565_BA9         1007743
5936_PFC_Nova    1004275
5893_PFC         1001385
5864_BA9          997339
5879_PFC_Nova     995605
5939_BA9          988669
6033_BA9          983178
5278_PFC          971907
5945_PFC          955434
5144_PFC          940984
5403_PFC          938094
5419_PFC          930869
5976_BA9          683775
5978_BA9          663300
Name: count, dtype: int64

- Save LIANA results

In [21]:
out_dir = '/projects/compbio/users/rche384/APE/Data/RDA/ASD/raw'

In [22]:
filtered_adata.write_h5ad(out_dir + "liana_res.h5ad")

... storing 'cluster' as categorical
... storing 'region' as categorical
... storing 'sex' as categorical
... storing 'diagnosis' as categorical
... storing 'Capbatch' as categorical
... storing 'Seqbatch' as categorical


# 3. Build Tensor

- Use expr_prod as the communication score function to construct tensor

In [26]:
filtered_adata

AnnData object with n_obs × n_vars = 62166 × 36745
    obs: 'cluster', 'sample', 'individual', 'region', 'age', 'sex', 'diagnosis', 'Capbatch', 'Seqbatch', 'post-mortem interval (hours)', 'RNA Integrity Number', 'genes', 'UMIs', 'RNA mitochondr. percent', 'RNA ribosomal percent', 'n_genes'
    var: 'n_cells'
    uns: 'log1p', 'liana_res'

In [27]:
tensor = li.multi.to_tensor_c2c(filtered_adata,
                                sample_key=sample_key,
                                score_key='expr_prod', # can be any score from liana
                                how='outer_cells',
                                outer_fraction=0.8)

100%|██████████| 23/23 [40:17<00:00, 105.09s/it]


In [28]:
tensor.tensor.shape

(23, 2555, 17, 17)

- Save constructed tensor

In [29]:
import os
out_path = '/projects/compbio/users/rche384/APE/Data/RDA/ASD/LIANA_expr_prod'
if not os.path.exists(out_path):
    os.makedirs(out_path)

In [30]:
from scipy.io import savemat
mymat={'c2ctensor':tensor.tensor}
savemat(out_path + "/c2ctensor.mat", mymat)

- Save dimension names (ligand-receptor pairs, sender cell types, receiver cell types)

In [32]:
pd.DataFrame(tensor.order_names[1]).to_csv(out_path + '/lr.txt', 
                      mode='w',
                      index=None,
                      header=None,
                      sep='\t')

pd.DataFrame(tensor.order_names[2]).to_csv(out_path + '/sender.txt', 
                      mode='w',
                      index=None,
                      header=None,
                      sep='\t')

pd.DataFrame(tensor.order_names[3]).to_csv(out_path + '/receiver.txt', 
                      mode='w',
                      index=None,
                      header=None,
                      sep='\t')

- Save individual-level information (e.g. disease status, age, gender) for subjects

In [33]:
subjects = tensor.order_names[0]
out_cols = ['sample', 'individual', 'age', 'sex', 'diagnosis', 'Seqbatch']
subject_info = filtered_adata.obs[out_cols].drop_duplicates().reset_index(drop=True)


# Step 1: Create a mapping from list values to order indices
order_mapping = {value: index for index, value in enumerate(subjects)}
# Step 2: Map the 'cov_ind' column to order indices
subject_info['order'] = subject_info['sample'].map(order_mapping)
# Step 3: Sort the DataFrame based on the 'order' column
subject_info = subject_info.sort_values('order').drop(columns='order').reset_index(drop=True)



In [34]:
subject_info

sample  individual  age sex diagnosis Seqbatch
0       4341_BA46        4341   13   M   Control      SB2
1        5144_PFC        5144    7   M       ASD      SB1
2        5278_PFC        5278   15   F       ASD      SB1
3        5294_BA9        5294   19   M       ASD      SB2
4        5387_BA9        5387   12   M   Control      SB2
5        5403_PFC        5403   16   M       ASD      SB1
6   5408_PFC_Nova        5408    6   M   Control      SB1
7        5419_PFC        5419   19   F       ASD      SB1
8        5531_BA9        5531   15   M       ASD      SB2
9   5538_PFC_Nova        5538   19   F   Control      SB1
10       5565_BA9        5565   12   M       ASD      SB2
11       5577_BA9        5577   21   M   Control      SB2
12       5841_BA9        5841   12   M       ASD      SB2
13       5864_BA9        5864   20   M       ASD      SB2
14  5879_PFC_Nova        5879   15   M   Control      SB1
15       5893_PFC        5893   19   M   Control      SB1
16  5936_PFC_Nova        5936   14   F   Control      SB1
17       5939_BA9        5939   21   M       ASD      SB2
18       5945_PFC        5945   20   M       ASD      SB1
19       5958_BA9        5958   22   M   Control      SB2
20       5976_BA9        5976    4   F   Control      SB2
21       5978_BA9        5978   11   M       ASD      SB2
22       6033_BA9        6033   14   F       ASD      SB2

In [35]:
subject_info.to_csv(out_path + '/subjects_info.txt', 
                    mode='w',
                    index=None,
                    header=True,
                    sep='\t')